# Задание 1

Исследовать структуру html-файлов, чтобы произвести парсинг всех данных. В каждом файле содержится информация об одном объекте из случайной предметной области. Полученные данные собрать и записать в json. Выполните также ряд операций с данными:

*	отсортируйте значения по одному из доступных полей
*	выполните фильтрацию по другому полю (запишите результат отдельно)
*	для одного выбранного числового поля посчитайте статистические характеристики (сумма, мин/макс, среднее и т.д.)
*	для одного текстового поля посчитайте частоту меток


In [54]:
from bs4 import BeautifulSoup
import pandas as pd
import json

In [55]:
def handle_file(path):

    with open(path, "r", encoding = "utf-8") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html.parser")
    # print(soup)

    product = soup.find_all("div", attrs={'class': "product-wrapper"})[0]
    # print(products)


    item = {}
    article_number = product.span.get_text().split("Артикул: ")[1].split("Наличие:")
    item['article_number'] = article_number[0].strip()
    item['availability'] = article_number[1].strip()
    item['name'] = product.find_all("h1", attrs={'class': 'title'})[0].get_text().split(":")[1].strip()
    item['city'] = soup.find('p', class_='address-price').text.split(':')[1].split()[0].strip().replace("Цена", "")
    item['price'] = int(soup.find('p', class_='address-price').text.split(':')[2].split()[0].strip())  # цена
    # city_price = product.p.get_text().split("Город: ")[1].split('Цена: ')
    # print(city_price)
    # item['city'] = city_price[0]
    # item['price'] = int(city_price[1].split(" ")[0].strip())
    item['color'] = product.find_all('span', attrs={'class': "color"})[0].get_text().split(":")[1].strip()
    item["quantity"] = int(product.find_all('span', attrs={'class': "quantity"})[0].get_text().split(":")[1].strip().split(" ")[0].strip())
    item['dimensions'] = soup.find_all('span')[3].text.split(':')[1].strip()  # размеры
    item['image_url'] = soup.find('img')['src']  # ссылка на картинку
    item['rating'] = float(soup.find_all('span')[4].text.split(':')[1].strip())
    item['views'] = int(soup.find_all('span')[5].text.split(':')[1].strip())

    return item

In [56]:
items = []
for i in range(76):
    items.append(handle_file(fr"C:\Users\ipavl\Учеба_УРФУ\1_курс\1_семестр\ИД\Practice_3\86\1\{i+2}.html"))
print(items)

[{'article_number': '561902-H-29808', 'availability': 'Нет', 'name': 'Moisturizer', 'city': 'Тбилиси', 'price': 1231, 'color': 'Желтый', 'quantity': 681, 'dimensions': '5x66x76', 'image_url': 'https://upload.products.ru/872873.webp', 'rating': 0.7, 'views': 87579}, {'article_number': '972220-X-10872', 'availability': 'Нет', 'name': 'Blueberry', 'city': 'Бильбао', 'price': 2446, 'color': 'Желтый', 'quantity': 178, 'dimensions': '49x76x76', 'image_url': 'https://upload.products.ru/633656.webp', 'rating': 1.3, 'views': 84561}, {'article_number': '563490-Y-46312', 'availability': 'Да', 'name': 'Blackberry', 'city': 'Луарка', 'price': 3375, 'color': 'Оранжевый', 'quantity': 152, 'dimensions': '9x37x84', 'image_url': 'https://upload.product.org/723763.jpg', 'rating': 2.9, 'views': 44222}, {'article_number': '128064-U-50096', 'availability': 'Да', 'name': 'Facial cleanser', 'city': 'Кадакес', 'price': 3850, 'color': 'Голубой', 'quantity': 387, 'dimensions': '31x94x84', 'image_url': 'https://u

In [57]:
df = pd.json_normalize(items)
df.head(5)

,article_number,availability,name,city,price,color,quantity,dimensions,image_url,rating,views
0,561902-H-29808,Нет,Moisturizer,Тбилиси,1231,Желтый,681,5x66x76,https://upload.products.ru/872873.webp,0.7,87579
1,972220-X-10872,Нет,Blueberry,Бильбао,2446,Желтый,178,49x76x76,https://upload.products.ru/633656.webp,1.3,84561
2,563490-Y-46312,Да,Blackberry,Луарка,3375,Оранжевый,152,9x37x84,https://upload.product.org/723763.jpg,2.9,44222
3,128064-U-50096,Да,Facial cleanser,Кадакес,3850,Голубой,387,31x94x84,https://upload.products.ru/393467.png,4.1,85437
4,335834-Y-33471,Нет,Raspberry,Баку,3219,Красный,72,77x49x40,https://upload.product.com/480984.svg,0.1,39964


Выведем статистические показатели для всех числовых признаков

In [58]:
df.describe()

,price,quantity,rating,views
count,76.000000,76.000000,76.000000,76.000000
mean,2574.907895,439.131579,2.444737,51038.934211
std,1366.274215,266.764083,1.476337,29032.306344
min,18.000000,5.000000,0.100000,2036.000000
25%,1360.750000,190.250000,1.300000,25075.250000
50%,2613.500000,448.000000,2.250000,48444.500000
75%,3693.750000,624.750000,3.700000,74428.750000
max,4957.000000,987.000000,5.000000,99591.000000


Статистические показатели для столбца views

In [59]:
print(f"Среднее значение: {df['views'].mean()}")
print(f"Максимальное значение: {df['views'].max()}")
print(f"Минимальное значение: {df['views'].min()}")
print(f"std: {df['views'].std()}")
print(f"Количество продукта: {df['views'].count()}")

Среднее значение: 51038.93421052631
Максимальное значение: 99591
Минимальное значение: 2036
std: 29032.306344179422
Количество продукта: 76


Отсортируем значения по одному из доступных полей

In [60]:
filter_1 = df.loc[(df["rating"] > 3) & (df["availability"] == 'Да')]
print(filter_1)

    article_number availability             name          city  price  \
3   128064-U-50096           Да  Facial cleanser       Кадакес   3850   
14  356415-O-14725           Да   Makeup remover        Малага   4482   
23  254995-B-48322           Да           Grapes        Семана   2864   
26  429289-E-70070           Да            Serum  Сьюдад-Реаль   3871   
36  249716-O-48426           Да      Night cream       Артейхо   4746   
41  754053-A-97331           Да          Shampoo       Гранада   1255   
45   96062-V-83179           Да        Baseboard     Хельсинки   4317   
49  943820-Q-41385           Да           Papaya         Сория   1457   
50   14834-C-25423           Да           Screws        Семана   3480   
51  891699-P-72074           Да           Banana       Мартоса   2014   
54  900693-V-52278           Да           Shovel      Будапешт   1026   
65  772332-T-96895           Да      Scaffolding         Афины   1069   
70  372498-R-18859           Да       Blackberry   

Выполним фильтрацию по другому полю

In [61]:
filter_2 = df.loc[(df['color'] == 'Зеленый') & (df['quantity'] > 600)]
print(filter_2)

    article_number availability         name     city  price    color  \
70  372498-R-18859           Да   Blackberry  Варшава   1633  Зеленый   
71  412846-X-76921           Да  Wheelbarrow   Овьедо   2343  Зеленый   

    quantity dimensions                              image_url  rating  views  
70       787   61x54x42  https://upload.all-prod.it/534593.jpg     3.8  85922  
71       671   87x24x15  https://upload.all-prod.it/685370.png     0.7  24737  


Для одного текстового поля посчитаем частоту меток

In [62]:
freq = df['color'].value_counts()
print(freq)

color
Желтый        19
Голубой       14
Синий         10
Зеленый       10
Фиолетовый    10
Красный        7
Оранжевый      6
Name: count, dtype: int64


Сохраним полученные данные в JSON формате

In [64]:
with open("first_task", "w", encoding = "utf-8") as f:
    json.dump(items, f, ensure_ascii=False)